In [1]:
import networkx as nx

In [2]:
def parseobo(obofile):
    lines = ""
    with open(obofile,"r") as f:
        for line in f:
            if line != "\n":
                lines += line
    lines = lines.split("[Term]\n")
    GOs = {}
    for line in lines[1:]:
        id = None
        interactions = {}
        line = line[:-1]
        data = line.split("\n")
        for data_ in data:
            key = data_.split(": ")[0]
            if key == "id":
                id  = data_.split(" ")[-1]
                GOs[id] = {"name": [], "alt_id":[], "namespace": [], "def": [], "synonym": [], "is_a":[], "negatively_regulates":[], "part_of":[], "positively_regulates":[], "regulates": [], "is_obsolete":[], "consider":[], "replaced_by": []}
            elif key == "relationship":
                #relationship: part_of GO:0005829 ! cytosol
                subindex = data_.split(": ")[1].split(" ")[0]
                relationGO = data_.split(": ")[1].split(subindex+" ")[-1].split(" !")[0] 
                GOs[id][subindex].append(relationGO)
            else:
                if key != "subset" and key != "xref" and key != "comment":
                    info = data_.split(": ")[1].split(" !")[0]
                    GOs[id][key].append(info.replace('"',""))

    return GOs

In [3]:
def makeGoGraph(GODict, drop_obsolete=True):
    G = nx.DiGraph()
    #parsing GO dict, adding first the nodes and then the graph
    for id in GODict:
        status_obsolete = False
        if len(GODict[id]["is_obsolete"]) > 0:
            status_obsolete = True

        if status_obsolete == False:
            G.add_node(id)
        else:
            if drop_obsolete == False:
                G.add_node(id)

    #adding interactions
    for id in G.nodes:
        for go in GODict[id]["is_a"]:
            G.add_edge(go, id)
        for go in GODict[id]["part_of"]:
            G.add_edge(go, id)
        for go in GODict[id]["negatively_regulates"]:
            G.add_edge(id, go)
        for go in GODict[id]["positively_regulates"]:
            G.add_edge(id, go)
        for go in GODict[id]["regulates"]:
            G.add_edge(id, go)
    return G

In [4]:
GODict = parseobo("go-basic.obo")

In [5]:
GONet = makeGoGraph(GODict)

In [6]:
nx.write_gml(GONet, "GO.gml")